In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("spark://de-zoomcamp.europe-west1-b.c.lexical-passkey-375922.internal:7077") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 14:54:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### We will try to reproduce the DBT model [`dm_monthly_zone_revenue.sql`](https://github.com/Valkea/DE_bootcamp/blob/main/week04/models/core/dm_monthly_zone_revenue.sql) using spark 

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*') # data/pq/green/all_years/all_months

In [4]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*') # data/pq/yellow/all_years/all_months

In [5]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



#### Let's find the columns with similar names

In [7]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

#### Let's find the columns with different names

In [8]:
set(df_green.columns) ^ set(df_yellow.columns)

{'ehail_fee',
 'lpep_dropoff_datetime',
 'lpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'tpep_pickup_datetime',
 'trip_type'}

#### Let's rename the dropoff and pickup columns

In [9]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [10]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

#### Let's check again 

In [11]:
set(df_green.columns) & set(df_yellow.columns)

{'DOLocationID',
 'PULocationID',
 'RatecodeID',
 'VendorID',
 'congestion_surcharge',
 'dropoff_datetime',
 'extra',
 'fare_amount',
 'improvement_surcharge',
 'mta_tax',
 'passenger_count',
 'payment_type',
 'pickup_datetime',
 'store_and_fwd_flag',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'trip_distance'}

In [12]:
set(df_green.columns) ^ set(df_yellow.columns)

{'ehail_fee', 'trip_type'}

#### Let's keep the original order

In [13]:
col_names = []

commun_cols = set(df_green.columns) & set(df_yellow.columns)
for col in df_green.columns:
    if col in commun_cols:
        col_names.append(col)
    
col_names

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

#### Let's try the selected column names

In [14]:
df_green.select(col_names).show(3)

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+
|       2|2020-01-23 13:10:15|2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.5|      2.05|        6.12|                  0.3|       44.97|           

#### Let's add a new column with the original dataset name

In [15]:
from pyspark.sql import functions as F

In [16]:
df_green_plus = df_green \
    .select(col_names) \
    .withColumn('service_type', F.lit("green"))

In [17]:
df_yellow_plus = df_yellow \
    .select(col_names) \
    .withColumn('service_type', F.lit("yellow"))

In [18]:
df_green_plus

DataFrame[VendorID: int, pickup_datetime: timestamp, dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: int, PULocationID: int, DOLocationID: int, passenger_count: int, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, payment_type: int, congestion_surcharge: double, service_type: string]

In [19]:
df_yellow_plus

DataFrame[VendorID: int, pickup_datetime: timestamp, dropoff_datetime: timestamp, store_and_fwd_flag: string, RatecodeID: int, PULocationID: int, DOLocationID: int, passenger_count: int, trip_distance: double, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, payment_type: int, congestion_surcharge: double, service_type: string]

#### Let's combine in order to get `trips_data`

In [20]:
df_trips_data = df_green_plus.unionAll(df_yellow_plus)

#### Let's use the combined dataframe

In [21]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



## Now we can do SQL requests

In [56]:
df_trips_data.createOrReplaceTempView('trips_data')

In [63]:
spark.sql("""
SELECT pickup_datetime, service_type
FROM trips_data
WHERE service_type = "yellow"
LIMIT 10
""").show()

+-------------------+------------+
|    pickup_datetime|service_type|
+-------------------+------------+
|2020-01-06 09:18:38|      yellow|
|2020-01-08 18:55:19|      yellow|
|2020-01-16 12:11:53|      yellow|
|2020-01-11 01:55:59|      yellow|
|2020-01-01 21:31:48|      yellow|
|2020-01-14 11:51:02|      yellow|
|2020-01-26 19:15:51|      yellow|
|2020-01-23 08:06:15|      yellow|
|2020-01-18 18:50:47|      yellow|
|2020-01-26 16:08:15|      yellow|
+-------------------+------------+



In [64]:
spark.sql("""
SELECT COUNT(1)
FROM trips_data
GROUP BY service_type
""").show()

+--------+
|count(1)|
+--------+
| 2304517|
|39649199|
+--------+



#### Then let's try to use the same SQL query as in the DBT model

In [77]:
df_result = spark.sql("""
    SELECT 
    -- Revenue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    -- SUM(ehail_fee) AS revenue_monthly_ehail_fee,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    -- count(tripid) AS total_monthly_trips,
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance

    FROM trips_data
    GROUP BY 1,2,3
""")

In [78]:
df_result.show()

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         250|2020-

#### Let's push data back to the data-lake (we pretend it is)

In [81]:
df_result.coalesce(1).write.parquet("data/report/revenue/", mode='overwrite') # df_result.coalesce(1) reduce partitions to 1